In [41]:
import streamlit as st
import yfinance as yf

def growth_rate_estimator(ticker, period):
    try:
        # Fetch historical data for the stock
        stock = yf.Ticker(ticker)
        history = stock.history(period=period)  # Adjust the period as needed

        # Calculate annualized growth rate (CAGR)
        initial_price = history["Close"].iloc[0]
        final_price = history["Close"].iloc[-1]
        num_years = len(history) / 252  # Assuming 252 trading days per year
        cagr = (final_price / initial_price) ** (1 / num_years) - 1

        return cagr
    except Exception as e:
        return f"Error estimating growth rate for {ticker}: {str(e)}"

def pe_ratio_estimator(ticker, growth_rate):
    try:
        # Get stock data
        stock = yf.Ticker(ticker)
        eps = stock.info['trailingEps']
        pe_ratio = stock.info['trailingPE']

        # Calculate intrinsic value
        intrinsic_value = eps * (1 + growth_rate) ** 5 * pe_ratio

        # Discount to present value (using 10% discount rate)
        discount_rate = 0.10
        present_value = intrinsic_value / (1 + discount_rate) ** 5

        return present_value
    except Exception as e:
        return f"Error: {str(e)}"

def get_stock_price(ticker):
    try:
        # Fetch stock data using yfinance
        stock = yf.Ticker(ticker)
        stock_history = stock.history(period="1d")  # Fetch historical data for 1 day

        # Extract the current stock price
        stock_price = stock_history["Close"].iloc[0]

        return stock_price
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"



ticker = 'AAPL'
period = "1y"
# “1d”, “5d”, “1mo”, “3mo”, “6mo”, “1y”, “2y”, “5y”, “10y”, “ytd”

print(f"Current stock price of {ticker}: ${get_stock_price(ticker):.2f}")
print(f"Expected growth rate for {ticker}: {growth_rate_estimator(ticker, period):.2%}")
print(f"Intrinsic stock value of {ticker}: ${pe_ratio_estimator(ticker, growth_rate_estimator(ticker, period)):.2f}")

Current stock price of AAPL: $169.65
Expected growth rate for AAPL: 2.99%
Intrinsic stock value of AAPL: $122.03


In [42]:
def dcf_estimator(ticker):
    try:
        # Get stock data
        stock = yf.Ticker(ticker)
        eps = stock.info['trailingEps']
        pe_ratio = stock.info['trailingPE']

        # Calculate historical CAGR (3-year period)
        historical_eps = [10, 12, 14]  # Example EPS data for years 1, 2, and 3
        num_years = len(historical_eps)
        cagr = ((historical_eps[-1] / historical_eps[0]) ** (1 / num_years)) - 1

        # Calculate intrinsic value
        expected_growth_rate = cagr
        intrinsic_value = eps * (1 + expected_growth_rate) ** 5 * pe_ratio

        # Discount to present value (using 10% discount rate)
        discount_rate = 0.10
        present_value = intrinsic_value / (1 + discount_rate) ** 5

        return present_value
    except Exception as e:
        return f"Error: {str(e)}"
    
# Calculate intrinsic value
result = dcf_estimator(ticker)
print(f"Intrinsic value of {ticker}: ${result:.2f}")


Intrinsic value of AAPL: $184.56


In [52]:
import yfinance as yf

def get_sector_and_market_cap(ticker):
    try:
        # Fetch stock data using yfinance
        stock = yf.Ticker(ticker)
        stock_info = stock.info

        # Extract sector and market cap
        sector = stock_info.get("sector")
        market_cap = stock_info.get("marketCap")

        return sector, market_cap
    except Exception as e:
        return f"Error fetching data for {ticker}: {str(e)}"

def calculate_total_market_cap(tickers):
    total_market_cap = 0
    for t in tickers:
        _, market_cap = get_sector_and_market_cap(t)
        if isinstance(market_cap, int):
            total_market_cap += market_cap
    return total_market_cap

# Example usage
sector, market_cap = get_sector_and_market_cap(ticker)

if sector and isinstance(market_cap, int):
    print(f"{ticker} belongs to the {sector} sector.")
    print(f"Market cap of {sector} sector: ${market_cap:.2f} (in billions)")
else:
    print(sector)

GOOGL belongs to the Communication Services sector.
Market cap of Communication Services sector: $1932177702912.00 (in billions)


In [ ]:
yf.Ticker('NVDA')

In [77]:
import time



# if sector and isinstance(market_cap, int) and isinstance(net_earnings, int):
print(f"{ticker} belongs to the {sector} sector.")
print(f"Market cap of {sector} sector: ${market_cap:.2f} (in billions)")
print(f"Total net earnings of {sector} sector: ${net_earnings:.2f} (in billions)")

AAPL belongs to the Technology sector.
Market cap of Technology sector: $2619718238208.00 (in billions)
Total net earnings of Technology sector: $96995000000.00 (in billions)


In [81]:
def get_sector_and_financials(ticker):
    try:
        # Fetch stock data using yfinance
        stock = yf.Ticker(ticker)
        stock_info = stock.info
        # time.sleep(0.02)
        stock_financials = stock.financials

        # Extract sector, market cap, and net earnings
        sector = stock_info.get("sector")
        market_cap = stock_info.get("marketCap")
        net_earnings = stock_financials.loc["Net Income"].iloc[0]

        return sector, market_cap, net_earnings
    except Exception as e:
        return f"Error fetching data for {ticker}: {str(e)}"

def calculate_total_market_cap_and_earnings(tickers):
    total_market_cap = 0
    total_earnings = 0
    for t in tickers:
        _, market_cap, net_earnings = get_sector_and_financials(t)
        if isinstance(market_cap, int) and isinstance(net_earnings, int):
            total_market_cap += market_cap
            total_earnings += net_earnings
    return total_market_cap, total_earnings

# Example usage
# ticker = input("Enter a stock ticker (e.g., AAPL): ")
sector, market_cap, net_earnings = get_sector_and_financials(ticker)

def calculate_market_pe_ratio(market_cap, net_earnings):
    try:
        market_pe_ratio = market_cap / net_earnings
        return market_pe_ratio
    except ZeroDivisionError:
        return "Error: Total net earnings are zero."

def pe_relative_estimator(ticker, market_pe_ratio):
    try:
        # Get stock data
        stock = yf.Ticker(ticker)
        eps = stock.info['trailingEps']

        # Calculate intrinsic value using P/E Relative approach
        intrinsic_value = eps * market_pe_ratio

        return intrinsic_value
    except Exception as e:
        return f"Error: {str(e)}"

intrinsic_value = pe_relative_estimator(ticker, calculate_market_pe_ratio(market_cap, net_earnings))
print(f"Intrinsic value of {ticker}: ${intrinsic_value:.2f}")

Intrinsic value of AAPL: $173.94
